In [38]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
from torchtext.vocab import GloVe
import torchtext
from torchtext.data import get_tokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [15]:
file_path = '../data/processed/full_2k.csv'
df = pd.read_csv(file_path)

In [51]:
le = LabelEncoder()
df['label'] = le.fit_transform(df['Category'])
num_classes = len(le.classes_)

df.head(5)

,Description,Category,cleaned_desc,label
0,The Russian author offers an affectionate chro...,Biography,the russian author offers an affectionate chro...,0
1,"""The Bible and the social and moral consequenc...",Religion,the bible and the social and moral consequence...,8
2,A New York Times Notable Book of the YearThis ...,Biography,a new york times notable book of the yearthis ...,0
3,"It was a wonderful summer, a great memory, the...",General,it was a wonderful summer a great memory the k...,3
4,"For the first time ever, veteran World War II ...",History,for the first time ever veteran world war ii c...,4


In [52]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return text

df['cleaned_desc'] = df['Description'].apply(clean_text)

In [53]:
# First, split off 10% test
X_temp, X_test, y_temp, y_test = train_test_split(
    df['cleaned_desc'], df['label'], 
    test_size=0.1, stratify=df['label'], random_state=13)

# Then split temp into train and val
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp,
    test_size=0.11111111111111,
    stratify=y_temp, random_state=13)

print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")

Train: 27543, Val: 3443, Test: 3443


In [54]:
glove = GloVe(name='6B', dim=50)

def sentence_to_vec(sentence, glove):
    words = sentence.split()
    vecs = [glove[word] for word in words if word in glove.stoi]
    if len(vecs) == 0:
        return torch.zeros(glove.dim)
    return torch.mean(torch.stack(vecs), dim=0)

# Convert all splits to vectors
X_train_vectors = torch.stack(X_train.apply(lambda x: sentence_to_vec(x, glove)).tolist())
X_val_vectors = torch.stack(X_val.apply(lambda x: sentence_to_vec(x, glove)).tolist())
X_test_vectors = torch.stack(X_test.apply(lambda x: sentence_to_vec(x, glove)).tolist())

# Convert labels to tensors
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

In [55]:
class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_loader = DataLoader(TextDataset(X_train_vectors, y_train_tensor), batch_size=32, shuffle=True)
val_loader = DataLoader(TextDataset(X_val_vectors, y_val_tensor), batch_size=32)
test_loader = DataLoader(TextDataset(X_test_vectors, y_test_tensor), batch_size=32)

NameError: name 'Dataset' is not defined